In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


## Dataset Import and Preparation

In [2]:
from datasets import load_dataset
dataset = load_dataset("babelscape/multinerd")


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 2678400
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 334800
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 335986
    })
})

In [4]:
from datasets import DatasetDict
# Filter out rows where language is 'en'
filtered_dataset = dataset.filter(lambda example: example['lang'] == 'en')

# PERSON(PER) 1&2, ORGANIZATION(ORG) 3&4, LOCATION(LOC) 5&6, DISEASES(DIS) 13&14, ANIMAL(ANIM) 7&8, Other(O) 0
required_classes = [1,2,3,4,5,6,7,8,13,14]
def remove_classes(row):
  row['ner_tags'] = [tag if tag in required_classes else 0 for tag in row['ner_tags']]
  return row
filtered_dataset = filtered_dataset.map(remove_classes)

Filter:   0%|          | 0/2678400 [00:00<?, ? examples/s]

Filter:   0%|          | 0/334800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/335986 [00:00<?, ? examples/s]

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

In [5]:
filtered_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32908
    })
})

In [6]:
import pandas as pd
pd.DataFrame(filtered_dataset['train'][:]).iloc[0]

tokens      [The, type, locality, is, Kīlauea, .]
ner_tags                       [0, 0, 0, 0, 5, 0]
lang                                           en
Name: 0, dtype: object

In [7]:
# Tag mapping
tag_mapping = {
    0: "O", 1: "B-PER", 2: "I-PER", 3: "B-ORG", 4: "I-ORG", 5: "B-LOC", 6: "I-LOC",
    7: "B-ANIM", 8: "I-ANIM", 9: "B-BIO", 10: "I-BIO", 11: "B-CEL", 12: "I-CEL",
    13: "B-DIS", 14: "I-DIS", 15: "B-EVE", 16: "I-EVE", 17: "B-FOOD", 18: "I-FOOD",
    19: "B-INST", 20: "I-INST", 21: "B-MEDIA", 22: "I-MEDIA", 23: "B-MYTH", 24: "I-MYTH",
    25: "B-PLANT", 26: "I-PLANT", 27: "B-TIME", 28: "I-TIME", 29: "B-VEHI", 30: "I-VEHI"
}

In [8]:
tag2label = tag_mapping
label2tag = {tag: idx for idx, tag in tag_mapping.items()}

In [9]:
tags = filtered_dataset['train'].features['ner_tags'].feature

In [10]:
def create_tag_names(batch):
  tag_name = {'ner_tags_str': [tag2label[idx] for idx in batch['ner_tags']]}
  return tag_name

In [11]:
filtered_dataset = filtered_dataset.map(create_tag_names)

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

In [13]:
import pandas as pd
pd.DataFrame(filtered_dataset['train'][:]).iloc[20]

tokens          [", G., rubens, ", also, has, occurred, alongs...
ner_tags        [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, ...
lang                                                           en
ner_tags_str    [O, O, O, O, O, O, O, O, O, B-ANIM, I-ANIM, O,...
Name: 20, dtype: object

## Model Building

In [14]:
from transformers import AutoTokenizer

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
tokenizer.is_fast

True

In [16]:

#helper function for tokenizing and formating the ner dataset --- reference - code used from https://huggingface.co/docs/transformers/v4.35.2/en/tasks/token_classification#evaluate
def format_and_tokenize(dataset_eng):
  tokenized_inputs = tokenizer(dataset_eng['tokens'],truncation=True,is_split_into_words=True)

  labels = []
  for i, label in enumerate(dataset_eng[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None

        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

  tokenized_inputs["labels"] = labels

  return tokenized_inputs
#format and tokenize the dataset
filtered_dataset = filtered_dataset.map(format_and_tokenize,batched=True)

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

In [17]:
filtered_dataset['train'][20]

{'tokens': ['"',
  'G.',
  'rubens',
  '"',
  'also',
  'has',
  'occurred',
  'alongside',
  '"',
  'G.',
  'firmus',
  '"',
  'in',
  'portions',
  'of',
  'its',
  'range',
  'and',
  'can',
  'be',
  'difficult',
  'to',
  'differentiate',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'lang': 'en',
 'ner_tags_str': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ANIM',
  'I-ANIM',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'input_ids': [101,
  1000,
  1043,
  1012,
  19469,
  2015,
  1000,
  2036,
  2038,
  4158,
  4077,
  1000,
  1043,
  1012,
  3813,
  2271,
  1000,
  1999,
  8810,
  1997,
  2049,
  2846,
  1998,
  2064,
  2022,
  3697,
  2000,
  21032,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

## Data Collation and Metrics

In [18]:
!pip install seqeval
!pip install evaluate

import evaluate
metric = evaluate.load('seqeval')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 758.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=4b9cab04abe6a6943faa5b4a229ebcdcbfcda90fc9528bece4ce8ade6a6ffafe
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [tag2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]

    true_labels = [[tag2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],

        "recall": results["overall_recall"],

        "f1": results["overall_f1"],

        "accuracy": results["overall_accuracy"],
    }



In [21]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(tag2label), id2label=tag2label, label2id=label2tag
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:

training_args = TrainingArguments(
    output_dir= './distilbert_system_A',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.003,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to=["none"]
)


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = filtered_dataset['train'],
    eval_dataset = filtered_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

In [23]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.016100,0.025468,0.928652,0.943077,0.935809,0.991401
2,0.007900,0.029986,0.935057,0.945515,0.940257,0.991794
3,0.004300,0.034900,0.936945,0.943842,0.940381,0.991845


TrainOutput(global_step=24615, training_loss=0.01622414468096544, metrics={'train_runtime': 5047.5065, 'train_samples_per_second': 156.053, 'train_steps_per_second': 4.877, 'total_flos': 1.2865955702769984e+16, 'train_loss': 0.01622414468096544, 'epoch': 3.0})

In [24]:
trainer.evaluate(eval_dataset=filtered_dataset['test'])


{'eval_loss': 0.025308331474661827,
 'eval_precision': 0.9374331321949758,
 'eval_recall': 0.9540089717346806,
 'eval_f1': 0.94564841996201,
 'eval_accuracy': 0.9916259635472948,
 'eval_runtime': 84.2538,
 'eval_samples_per_second': 390.582,
 'eval_steps_per_second': 12.213,
 'epoch': 3.0}